## Importing Libraries

In [14]:
import pandas as pd
import numpy as np                                      # for dealing with data
from scipy.signal import butter, sosfiltfilt, sosfreqz  # for filtering
import matplotlib.pyplot as plt                         # for plotting
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, auc
import os
from os import listdir
from os.path import isfile, join, isdir

## Load data

In [31]:
train_data_list = np.load('./data/train_data.npy')
test_data_list = np.load('./data/test_data.npy')
print('Epoched training data shape: ' + str(train_data_list.shape)) #! (16, 340, 56, 140)
print('Epoched training data: ' , train_data_list)
print('Epoched testing data shape: ' + str(test_data_list.shape)) #! (10, 340, 56, 140)
print('Epoched testing data: ' , test_data_list)
Y_true_Labels_for_test = np.reshape(pd.read_csv('./data/true_labels.csv', header=None).values, 3400)
# Shape: (3400,)
print('Y_true_Labels_for_test: ', Y_true_Labels_for_test.shape)
unique, counts = np.unique(Y_true_Labels_for_test, return_counts=True)
print("Class distribution:")
for label, count in zip(unique, counts):
    print(f"Class {label}: {count}")
    
subj, trial, numChannel, sample = train_data_list.shape #! (16, 340, 30, 140)
X_train_valid = np.reshape(train_data_list, (-1, 1, numChannel, sample)) #! (5440, 1, 30, 140)
X_test = np.reshape(test_data_list, (-1, 1, numChannel, sample)) #! (3400, 1, 30, 140)

Y_train_valid = pd.read_csv('data/TrainLabels.csv')['Prediction'].values

print('subject: ', subj)
print('trial: ', trial)
print('numChannel: ', numChannel)
print('sample: ', sample)
print('X_train_valid: ', X_train_valid.shape)
print('X_test: ', X_test.shape)
print('Y_train_valid: ', Y_train_valid.shape)

Epoched training data shape: (16, 340, 30, 140)
Epoched training data:  [[[[-1.36170070e+01  8.57873395e+00 -1.01538360e+01 ... -3.12661300e+01
    -4.96768840e+01 -2.62316980e+01]
   [-1.55638728e+01  1.08542292e+01 -1.17740148e+01 ... -3.44873038e+01
    -6.00591448e+01 -3.83019438e+01]
   [-4.49941225e+00  3.68569075e+00 -4.99533225e+00 ... -1.79836743e+01
    -3.75608613e+01 -1.29955443e+01]
   ...
   [-1.50661359e+01  2.54798811e+01 -9.40860585e+00 ... -1.49962885e+00
    -2.43487779e+01  1.07817581e+01]
   [-5.35187733e+01  3.00747677e+01  3.38864697e+01 ...  1.07840547e+01
     2.37090777e+01 -1.15545703e+01]
   [-9.79558657e+01  5.70724023e+01  9.24201823e+01 ...  2.87769053e+01
     7.88170803e+01 -3.48479577e+01]]

  [[ 2.40449131e+01  2.53053610e+00  2.59682721e+01 ... -1.87883359e+01
     8.42658410e+00 -1.44918759e+01]
   [ 2.52209564e+01 -5.92808165e+00  1.52453164e+01 ... -1.31449896e+01
     8.38659535e+00 -1.38532356e+01]
   [ 2.52776921e+01  1.07454371e+01  3.57564315

## EEGNet

In [32]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K

def EEGNet(nb_classes, Chans = 56, Samples = 128,
             dropoutRate = 0.5, kernLength = 64, F1 = 8,
             D = 2, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout'):
    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
      raise ValueError('dropoutType must be one of SpatialDropout2D or Dropout, passed as a string.')

    input   = Input(shape = (1, Chans, Samples))

    block1   = Conv2D(F1, (1, kernLength), padding = 'same', use_bias = False)(input)
    block1   = BatchNormalization(axis = 1)(block1)
    block1   = DepthwiseConv2D((Chans, 1), use_bias = False, depth_multiplier = D, depthwise_constraint = max_norm(1.),
                                  data_format='channels_first')(block1)
    block1   = BatchNormalization(axis = 1)(block1)
    block1   = Activation('elu')(block1)
    block1   = AveragePooling2D((1, 4), data_format='channels_first')(block1)
    block1   = dropoutType(dropoutRate)(block1)

    block2   = SeparableConv2D(F2, (1, 16), use_bias = False, padding = 'same')(block1)
    block2   = BatchNormalization(axis = 1)(block2)
    block2   = Activation('elu')(block2)
    block2   = AveragePooling2D((1, 8), data_format='channels_first')(block2)
    block2   = dropoutType(dropoutRate)(block2)

    flatten   = Flatten(name = 'flatten')(block2)

    dense    = Dense(nb_classes, name = 'dense',
                         kernel_constraint = max_norm(norm_rate))(flatten)
    softmax   = Activation('softmax', name = 'softmax')(dense)

    return Model(inputs=input, outputs=softmax)


## Some helper function

In [33]:
def format_results(title, results):
    formatted = " ".join(f"{val:.4f}" for val in results)
    print(f"{title}: {formatted}")

def print_metric_section(title, metrics_dict, format_str="{:<25} : {:.4f}"):
    print(f"\n{title}")
    print('-' * 50)
    for metric, value in metrics_dict.items():
        print(format_str.format(metric, value))
    print('-' * 50)

## Classification

In [ ]:
from sklearn.model_selection import train_test_split
import keras
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import f1_score, balanced_accuracy_score, accuracy_score, precision_score, recall_score, roc_auc_score

import random
import tensorflow as tf
import pandas as pd
import numpy as np

# Define lists to store results
accuracy_results = []
accuracy_results_balanced = []
precision_results = []
recall_results = []
f1_results = []
auc_result = []

# Number of model runs
n_runs = 10

for run in range(n_runs):
    print(f"Running iteration {run + 1}/{n_runs}...")
    state_seed = random.randint(1, 99999)

    
    # Re-split training/validation set (ensure different random allocation)
    print(X_train_valid.shape, Y_train_valid.shape)
    X_train, X_valid, Y_train, Y_valid = train_test_split(
        X_train_valid, Y_train_valid, test_size=0.25, stratify=Y_train_valid, random_state=state_seed
    )

    # Calculate class_weights
    class_weights = compute_class_weight(
        class_weight='balanced', classes=np.unique(Y_train), y=Y_train
    )
    class_weights = dict(enumerate(class_weights))  # Convert to dictionary format
    print("Class Weights:", class_weights)

    # Initialize and train model
    model = EEGNet(nb_classes=2, Chans=numChannel, Samples=sample, dropoutRate=0.5)
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # Set ModelCheckpoint
    checkpointer = ModelCheckpoint(filepath=f'/tmp/checkpoint_{run}.keras', verbose=0, save_best_only=True, monitor='val_loss', mode='min')
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True, verbose=1)
    
    # Define ReduceLROnPlateau
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',       # Monitor the metric
        factor=0.3,               # Learning rate decrease factor
        patience=10,              # Number of epochs to wait before reducing learning rate
        min_lr=1e-6,              # Minimum learning rate
        verbose=1                 # Show messages
    )
    
    fittedModel = model.fit(X_train, Y_train, batch_size=32, epochs=150, verbose=2,
            validation_data=(X_valid, Y_valid),
            callbacks=[checkpointer, early_stopping], class_weight=class_weights)

    # Load best weights
    model.load_weights(f'/tmp/checkpoint_{run}.keras')

    # Output layer: softmax
    print(X_test.shape, Y_true_Labels_for_test.shape) 
    y_probs = model.predict(X_test)[:, 1]  # 取得類別 1 的機率
    y_pred = model.predict(X_test).argmax(axis=-1)

    # Calculate metrics
    accuracy = accuracy_score(Y_true_Labels_for_test, y_pred)
    accuracy_balanced = balanced_accuracy_score(Y_true_Labels_for_test, y_pred)
    precision = precision_score(Y_true_Labels_for_test, y_pred)
    recall = recall_score(Y_true_Labels_for_test, y_pred)
    f1 = f1_score(Y_true_Labels_for_test, y_pred)
    auc = roc_auc_score(Y_true_Labels_for_test, y_probs)  # Use probability of positive class
    
    # Save results
    accuracy_results.append(accuracy)
    accuracy_results_balanced.append(accuracy_balanced)
    precision_results.append(precision)
    recall_results.append(recall)
    f1_results.append(f1)
    auc_result.append(auc)

# Calculate mean and std
accuracy_mean, accuracy_std = np.mean(accuracy_results), np.std(accuracy_results)
accuracy_mean_balanced, accuracy_std_balanced = np.mean(accuracy_results_balanced), np.std(accuracy_results_balanced)
precision_mean, precision_std = np.mean(precision_results), np.std(precision_results)
recall_mean, recall_std = np.mean(recall_results), np.std(recall_results)
f1_mean, f1_std = np.mean(f1_results), np.std(f1_results)
auc_mean, auc_std = np.mean(auc_result), np.std(auc_result)

# now get min/max directly from each list
max_accuracy             = max(accuracy_results)
min_accuracy             = min(accuracy_results)
max_balanced_accuracy    = max(accuracy_results_balanced)
min_balanced_accuracy    = min(accuracy_results_balanced)
max_precision            = max(precision_results)
min_precision            = min(precision_results)
max_recall               = max(recall_results)
min_recall               = min(recall_results)
max_f1                   = max(f1_results)
min_f1                   = min(f1_results)
max_auc                  = max(auc_result)
min_auc                  = min(auc_result)

# 顯示結果
print('\nBasic Metrics')
print('-' * 50)
print(accuracy_results)
print(accuracy_results_balanced)
print(precision_results)
print(recall_results)
print(f1_results)
print(auc_result)
print('---------------------------------------------------------------')
print('\nMean and Standard Deviation')
mean_std_metrics = {
    'Accuracy': (accuracy_mean, accuracy_std),
    'Balanced Accuracy': (accuracy_mean_balanced, accuracy_std_balanced),
    'Precision': (precision_mean, precision_std),
    'Recall': (recall_mean, recall_std),
    'F1': (f1_mean, f1_std),
    'AUC': (auc_mean, auc_std)
}
print('-' * 50)
for metric, (mean, std) in mean_std_metrics.items():
    print(f"{metric:<25} : Mean = {mean:.4f} ± {std:.4f}")
print('-' * 50)
max_metrics = {
    'Max Accuracy': max_accuracy,
    'Max Balanced Accuracy': max_balanced_accuracy,
    'Max Precision': max_precision,
    'Max Recall': max_recall,
    'Max F1': max_f1,
    'Max AUC': max_auc
}
print_metric_section("Maximum Values", max_metrics)
min_metrics = {
    'Min Accuracy': min_accuracy,
    'Min Balanced Accuracy': min_balanced_accuracy,
    'Min Precision': min_precision,
    'Min Recall': min_recall,
    'Min F1': min_f1,
    'Min AUC': min_auc
}
print_metric_section("Minimum Values", min_metrics)

Running iteration 1/10...
(5440, 1, 30, 140) (5440,)
Class Weights: {0: 1.7114093959731544, 1: 0.7063711911357341}
Epoch 1/150
128/128 - 3s - 20ms/step - accuracy: 0.4365 - loss: 0.6924 - val_accuracy: 0.4221 - val_loss: 0.6933 - learning_rate: 0.0010
Epoch 2/150
128/128 - 1s - 8ms/step - accuracy: 0.4510 - loss: 0.6900 - val_accuracy: 0.4566 - val_loss: 0.6911 - learning_rate: 0.0010
Epoch 3/150
128/128 - 1s - 8ms/step - accuracy: 0.4924 - loss: 0.6837 - val_accuracy: 0.5221 - val_loss: 0.6864 - learning_rate: 0.0010
Epoch 4/150
128/128 - 1s - 9ms/step - accuracy: 0.5370 - loss: 0.6817 - val_accuracy: 0.5868 - val_loss: 0.6806 - learning_rate: 0.0010
Epoch 5/150
128/128 - 1s - 8ms/step - accuracy: 0.5897 - loss: 0.6683 - val_accuracy: 0.6478 - val_loss: 0.6579 - learning_rate: 0.0010
Epoch 6/150
128/128 - 1s - 9ms/step - accuracy: 0.5936 - loss: 0.6644 - val_accuracy: 0.6316 - val_loss: 0.6534 - learning_rate: 0.0010
Epoch 7/150
128/128 - 1s - 11ms/step - accuracy: 0.6262 - loss: 0.66